# Docker容器集群

真实世界运行的系统不会像我们之前举例的那么简单，往往需要多个服务相互配合对外提供服务。接下来，我们来运行一个真实的案例。

## 最简单的博客系统

[Flaskr](https://github.com/pallets/flask/tree/master/examples/flaskr)是一款基于Flask的博客系统，我们将用它来演示如何使用多个容器组成集群，对外提供服务。

我们来看看这个系统的源码结构

In [ ]:
!ls flaskr

它是一个非常简单的Flask程序，`flaskr.py`是主程序，`templates`存放了模板文件，`static`存放了静态资源。

接着让我们把它打包成Docker镜像

In [ ]:
!docker build -t flaskr flaskr/

然后让我们把它跑起来

In [ ]:
!docker run --name=flaskr -d -p 30080:5000 flaskr

通过浏览器访问 `http://$SERVER_IP:30080` 我们可以打开这个博客系统

## Nginx

在真实环境中，我们不会把Flask系统直接暴露到公网，而会在前面加上Nginx，用它来处理静态文件，同时将动态请求在多个后端服务之间做负载均衡。让我们将Nginx加入集群。

还是先来看看Nginx镜像的源码

In [ ]:
!ls nginx

接着把它打包成Docker镜像

In [ ]:
!docker build -t flaskr-nginx nginx/

前面的操作都跟之前操作单个镜像的流程相同，接下来我们将刚才生成的两个镜像组成集群对外提供服务。

## 容器网络

将多个容器组成集群的难点在于每一个容器都运行在隔离环境中，具有不同的IP地址，而IP地址是临时分配，并不是固定的，我们需要想办法让Nginx容器找到Flaskr容器，这样Nginx才能顺利的将动态请求发往Flaskr。

这个问题从Docker诞生就存在，程序员设计了多种不同的办法来解决它。这里我们将使用最新版本的Docker 1.10之后提供的网络功能来解决它。

首先为我们的容器集群创建一个新的隔离网络

In [ ]:
!docker network create app

然后删除掉之前的flaskr容器，将它在新的隔离网络中运行起来

In [ ]:
!docker rm -f flaskr

In [ ]:
!docker run --name=flaskr -d --net=app --volume /app/static flaskr

接着我们运行nginx容器，使用相同的容器网络

In [ ]:
!docker run --name=flaskr-nginx -d --net=app -p 30080:80 --volumes-from=flaskr flaskr-nginx

再使用浏览器访问 `http://$SERVER_IP:30080`，我们可以正常打开

让我们来看看是如何做到的。

因为两个容器运行在同一个隔离网络中，它们可以使用容器名称作为域名找到对方。所以我们在nginx配置了固定的flaskr地址

In [ ]:
!cat nginx/flaskr.conf

这样利用Docker提供的域名解析功能，我们将两个容器连接在了一起。

也许你会注意到前面指令当中的 `--volume /app/static` 以及 `--volumes-from=flaskr`，它解决了容器集群的第二个难点：如何在多个隔离的容器当中共享文件。这里我们采用的是数据卷的办法。

## 数据卷

Docker容器在销毁时，会删除所有容器内做的修改。为了持久化保存容器内的文件，可以使用数据卷。Docker支持将一个数据卷挂载到容器的指定路径。默认支持的数据卷格式有：

- 主机指定目录或文件
- Docker自动管理的目录

通过使用第三方的数据卷驱动，还支持更多的服务。

我们使用在运行`docker run`的时候传入的`--volume /app/static`参数让Docker创建一个自动管理的数据卷，将它加载到`/app/static`目录下，并将这个目录中的文件放入数据卷中。

`--volumes-from=flaskr` 参数让nginx容器在相同的位置加载flaskr容器所加载的数据卷。这样两个隔离的容器通过使用相同的数据卷完成了静态文件共享。

让我们来检查一下nginx确实有正常的处理静态文件。

In [ ]:
!docker logs flaskr

In [ ]:
!docker logs flaskr-nginx

可以看到，在flaskr容器中已经没有在处理静态资源，而nginx容器在提供静态资源。

## 总结

通过刚才的实验，我们完成了

- 使用容器网络连接多个隔离容器
- 使用数据卷在容器之间共享静态资源